# Masking data


## Description

In this notebook I will demonstrate how to mask one ImageCollection with another image. This is a very common piece of analysis in remote sensing.

We will:
1. Load a shapefile (a common vector file format) delineating the Murray-Darling-Basin (I found this shapefile online), and convert it to a geojson.
2. Load a timeseries of MODIS NDVI within the limits of the geojson
3. Load the Global Forest Change product, thesholding it to create a 'mask' of trees/not-trees (AKA a binary mask)
4. Mask the MODIS NDVI product with the 'trees' mask.
5. Extract the time-series and plot.

## Load packages

Import Python packages that are used for the analysis.


In [ ]:
%matplotlib inline

import ee
import eemont
import geemap as gmap
import geemap.colormaps as cm
import pandas as pd
import numpy as np
import seaborn as sb
import geopandas as gpd
from matplotlib import pyplot as plt

### Connect to Google Earth Engine (GEE)

In [ ]:
Map = gmap.Map()

***

## Open the shapefile and convert to geojson

In [ ]:
# #open file
# gdf = gpd.read_file('/scratch/du53/cb3058/mdb_boundary.shp')

# #convert to wgs84 (only projectioj that works with geojson)
# gdf = gdf.to_crs('epsg:4326')

# #export to file for use later
# gdf.to_file('/scratch/du53/cb3058/mdb_boundary.geojson')

## Load MODIS NDVI ImageCollection over a region specified by the MDB


In [ ]:
## open the goejson and convert to ee object
geometry = gmap.geojson_to_ee('/scratch/du53/al0850/project/data/mdb_boundary.geojson')
Map.centerObject(geometry);

In [ ]:
#load the MODIS NDVI product
ndvi = (ee.ImageCollection('MODIS/061/MOD13A1')
                  .filterBounds(geometry) # filter to MDB
                  .map(lambda image: image.clip(geometry)) #clip the data to MDB
                  .filterDate('2017-01-01', '2022-12-31') #timeseries
                  .preprocess() #this will do the rescaling
                  .select('NDVI') #just select the NDVI band
     )

#define a visualization dict
NDVI_vis = {'min':0, 'max':1,
           'palette':list(cm.palettes.ndvi),
           }

# Plot the first time-step
Map.addLayer(ndvi.first(), NDVI_vis, 'NDVI')
Map.add_colorbar(NDVI_vis['palette'],caption='NDVI')
Map.addLayerControl()
Map

## Load the GFC treecover layer and create a mask

In [ ]:
#load gfc
gfc = (ee.Image('UMD/hansen/global_forest_change_2022_v1_10')
       .select('treecover2000')
       .clip(geometry)
      )

# create a binary mask where treecover is greater than 25 %
trees_mask = gfc.gt(25)


## Now mask the NDVI data with the 'tree mask' and plot

In [ ]:
#mask ndvi so only places with trees remain
ndvi = ndvi.map(lambda image: image.updateMask(trees_mask))

# Plot again, including the tree mask (mask trees_mask with itself so 0's don't plot)
Map.addLayer(trees_mask.updateMask(trees_mask),
    {'palette': ['000000', '00FF00'], 'max': 1}, 'Trees');

#add the long-term mean ndvi to the map
Map.addLayer(ndvi.mean(), NDVI_vis, 'NDVI')
Map

## Generate a time-series for trees in the MDB

### Calculate the time series for trees in MDB

- `Image Collection`: MODIS NDVI
- `bands` NDVI
- `geometry`: the MDB
- `reducer` staistics to compute over the x,y dimension: e.g. minimum, mean, maximum, median
- `scale`: This MODIS product has a resolution of 500m



In [ ]:
ts = ndvi.getTimeSeriesByRegion(geometry = geometry,
                              bands = ['NDVI'], #which bands are we summarising?
                              reducer = [ee.Reducer.mean()], #lets take the mean of the pixels
                              scale = 500)

### Convert the time-series to a Pandas Dataframe

In [ ]:
# convert to a pandas dataframe
ts_df = gmap.ee_to_pandas(ts)

#convert dat column to datetime
ts_df['date'] = pd.to_datetime(ts_df['date'],
                               infer_datetime_format = True)

#make the index of dataframe the time stamps, and further clean up df
ts_df = ts_df.set_index('date').drop('reducer', axis=1)

ts_df.head() #print the top 5 rows

### Plot

In [ ]:
#now plot
ts_df.plot(figsize = (11,4))
plt.title('NDVI of Trees over the MDB');
plt.ylabel('NDVI');

***